In [1]:
# %env SM_FRAMEWORK=tf.keras
import zipfile, os, numpy as np, pickle, yaml, gc, tensorflow as tf
import segmentation_models as sm
import tensorflow_addons as tfa
sys.path.append("..")
from keras import backend as K
K.clear_session()
from sklearn.metrics import jaccard_score
from segmentation_models import Unet
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
class_type = 0 # 0=NL, 1=AP

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
# path test
top_layer_path_train = '../my_data/datasets/all_seg_data_NL_zm_2/'
top_layer_path = '../my_data/datasets/test_data_NL/'
# S1 [img, msk]
S1_img_stack = ['T3_image_arr_384_valid.npy', 'T3_masks_arr_384_valid.npy', 'T3_image_arr_384_train.npy', 'T3_masks_arr_384_train.npy']
# save path
# save_path = f'results_log/S1/all_S1-test_results_{datatype}'


# S1 model weight path
S1_weights=['../checkpoints/S1-segment/2DResNet50-384-epochs_100-lr_0.001-batch_8-2022.04.07-original_3T_NL-data_FTL-20--sample.hdf5',
                            '../checkpoints/S1-segment/2DResNet50-384-epochs_100-lr_0.001-batch_8-2022.04.07-original_3T_NL-data_FTL-50--sample.hdf5',
                            '../checkpoints/S1-segment/2DResNet50-384-epochs_100-lr_0.001-batch_8-2022.04.07-original_3T_NL-data_FTL-100--sample.hdf5',
                            '../my_data/my_weights/S1_ResNet.hdf5',
                            '../my_data/my_weights/S1_DenseNet.hdf5',
                            '../my_data/my_weights/S1_VGGNet.hdf5']
# S1_weights=['./checkpoints/S1-segment/best-valid-auc_2DDenseUnet-12.16-original_3T_NL-data_dice-loss.hdf5', './checkpoints/S1-segment/best-valid-auc_2DResNet50-12.16-original_3T_NL-data_dice-loss.hdf5', config["S1_vgg"]]
S1_backbone = ['resnet50', 'resnet50', 'resnet50', 'resnet50','densenet121','vgg16']


In [3]:
def S1_dataloader(valid_data='Mix', tune_type='test', data_len=None):
    if tune_type=='test':
        img_layer_path = top_layer_path
        if valid_data == '3.0T':
            # loading valida data 3.0T + 1.5T: image / masks
            X_valid = np.load(img_layer_path +'/'+ S1_img_stack[0])
            y_valid = np.load(img_layer_path +'/'+ S1_img_stack[1])
    elif tune_type=='train':
        img_layer_path = top_layer_path_train
        if data_len == 999:
            X_valid = np.load(img_layer_path +'/'+ S1_img_stack[2])
            y_valid = np.load(img_layer_path +'/'+ S1_img_stack[3])
        else:
            X_valid = np.load(img_layer_path +'/'+ S1_img_stack[2])[0: data_len]
            y_valid = np.load(img_layer_path +'/'+ S1_img_stack[3])[0: data_len]
        
    X_valid = np.reshape(X_valid, (X_valid.shape[0]*32,384,384,1))
    y_valid = np.reshape(y_valid, (y_valid.shape[0]*32,384,384,1))
    return X_valid.astype(np.float32), y_valid.astype(np.float32)

In [4]:
def S1_model_loader(weight_path, backbone, mode, data_len=None):
    S1_X_valid, S1_y_valid= S1_dataloader(valid_data='3.0T', tune_type=mode, data_len=data_len)
    print(f'S1 data shape: img {S1_X_valid.shape} msk {S1_y_valid.shape}')
    model = Unet(backbone, encoder_weights=None, input_shape=(None, None, 1))
    model.load_weights(weight_path)
    Results = model.predict(S1_X_valid, batch_size=1, verbose=1)
    return np.array(Results), np.array(S1_X_valid), np.array(S1_y_valid)

In [5]:
from sklearn.metrics import confusion_matrix
S1_thresholds = 0.5
S1_pred_stack=[]
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f* y_pred_f)
    dice = (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))
    return dice
def iou(t, p):
    # ytrue, ypred is a flatten vector
    y_pred = t.flatten()
    y_true = p.flatten()
    current = confusion_matrix(y_true.astype(np.int8), y_pred.astype(np.int8), labels=[0, 1])
    # compute mean iou
    intersection = np.diag(current)
    ground_truth_set = current.sum(axis=1)
    predicted_set = current.sum(axis=0)
    union = ground_truth_set + predicted_set - intersection
    IoU = intersection / union.astype(np.float32)
    return np.mean(IoU)

class metric_:
    def __init__(self,y_true, y_pred):
        self.y_true_f = K.flatten(y_true)
        self.y_pred_f = K.flatten(y_pred)
        # self.current = confusion_matrix(K.cast(self.y_true_f, dtype='int8'), K.cast((self.y_pred_f>0.5), dtype='int8'), labels=[0, 1])
        self.tn, self.fp, self.fn, self.tp = confusion_matrix(K.cast(self.y_true_f, dtype='int8'), K.cast((self.y_pred_f>0.5), dtype='int8'), labels=[0, 1]).ravel()
        # self.current = (tn, fp, fn, tp)
    def dice_coef(self):
        # intersection = K.sum(self.y_true_f* self.y_pred_f )
        # intersection = np.diag(self.current)
        # ground_truth_set = self.current.sum(axis=1)
        # predicted_set = self.current.sum(axis=0)
        # # dice = (2. * intersection) / (K.sum(self.y_true_f) + K.sum(self.y_pred_f ))
        # dice = (2. * intersection) / (ground_truth_set + predicted_set)
        dice = (2.*self.tp) / (self.tp + self.fn + self.tp + self.fp)
        # return np.mean(dice)
        return dice
        
    def iou(self):
        # ytrue, ypred is a flatten vector
        # compute mean iou
        # intersection = np.diag(self.current)
        # ground_truth_set = self.current.sum(axis=1)
        # predicted_set = self.current.sum(axis=0)
        # union = ground_truth_set + predicted_set - intersection
        # IoU = intersection / union.astype(np.float32)
        IoU = self.tp / (self.tp + self.fn + self.fp)
        # return np.mean(IoU)
        return IoU
    def acc(self):
        # tn, fp, fn, tp = self.current .ravel()
        # return ((tp+tn)/(tp+fp+fn+tn))
        return ((self.tp+self.tn)/(self.tp+self.fp+self.fn+self.tn))
        # "Accuracy: "+str(round((tp+tn)/(tp+fp+fn+tn), 3))

In [6]:
# ----S1 model test 1 - 3----
mode_list=['test']
train_data_len = [20,50,100,999,999,999]
for idx,d in enumerate(mode_list):
    print(f'Start model = {d}')
    for i in range(6):
        if train_data_len[i]==999:
            print(f'Train Sample = 140 - {S1_backbone[i]}')
        else:
            print(f'Train Sample = {train_data_len[i]} - {S1_backbone[i]}')
        if d == 'train':
            S1_pred, S1_X_valid, S1_y_valid = S1_model_loader(S1_weights[i], S1_backbone[i], d, data_len=train_data_len[i])
        else:
            S1_pred, S1_X_valid, S1_y_valid = S1_model_loader(S1_weights[i], S1_backbone[i], d)
        metric_class = metric_(S1_y_valid,S1_pred)
        print("Dice: ", metric_class.dice_coef())
        print("IoU:  ", metric_class.iou())
        print("Acc:  ", metric_class.acc())
        # S1_pred_stack.append([S1_pred, S1_y_valid])
        # print("Dice: ", dice_coef(S1_y_valid,S1_pred).numpy())
        # print("IoU:  ", iou(S1_y_valid,S1_pred))

Start model = test
Train Sample = 20 - resnet50
S1 data shape: img (1440, 384, 384, 1) msk (1440, 384, 384, 1)
1440/1440 [==============================] - 45s 31ms/step
Dice:  0.6684784099073653
IoU:   0.502040984450623
Acc:   0.9994306587878569
Train Sample = 50 - resnet50
S1 data shape: img (1440, 384, 384, 1) msk (1440, 384, 384, 1)
1440/1440 [==============================] - 43s 30ms/step
Dice:  0.7752051030470006
IoU:   0.6329264638312324
Acc:   0.999567323849525
Train Sample = 100 - resnet50
S1 data shape: img (1440, 384, 384, 1) msk (1440, 384, 384, 1)
1440/1440 [==============================] - 43s 30ms/step
Dice:  0.7827925442234469
IoU:   0.6431052821016787
Acc:   0.9995843910876615
Train Sample = 140 - resnet50
S1 data shape: img (1440, 384, 384, 1) msk (1440, 384, 384, 1)
1440/1440 [==============================] - 42s 29ms/step
Dice:  0.8281570127352231
IoU:   0.7067132898650882
Acc:   0.9996322490550854
Train Sample = 140 - densenet121
S1 data shape: img (1440, 384, 3